In [ ]:
#Final Project: Fake news detection using deep learning
#Done By: Mohammed A. M. Salem
#Supervisor: Dr. Motaz Saad

In [ ]:
import numpy as np
import os
import time
import io #codecs
import pandas as pd
import regex as re
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#Deep learning libraries
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sanaousty","key":"6ca4bdb2ed7688aeddcd8c127e52b14d"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 88% 36.0M/41.0M [00:01<00:00, 11.8MB/s]
100% 41.0M/41.0M [00:01<00:00, 24.3MB/s]


In [ ]:
!ls

fake-and-real-news-dataset.zip	kaggle.json  sample_data


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("fake-and-real-news-dataset.zip", "r")
zip_ref.extractall("files")
zip_ref.close()

In [ ]:
import pandas as pd
fake = pd.read_csv("/content/files/Fake.csv")
true = pd.read_csv("/content/files/True.csv")

In [ ]:
# Add flag to track fake and real
fake['target'] = 1 #indicates to fake news
true['target'] = 0 #indicates to real ro true news

In [ ]:
#combination of the two datasets
data =pd.concat([true,fake], ignore_index=True)

In [ ]:
#check null or missing values
data.isnull().sum()

title      0
text       0
subject    0
date       0
target     0
dtype: int64

In [ ]:
#check deplicates records
data.drop_duplicates()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [ ]:
#remove unnecessary features
data.drop(['title'], axis =1, inplace= True)
data.drop(['date'], axis =1, inplace= True)

In [ ]:
#preprocessing phase and create a corpus
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
corpus = []

for i in range(0, len(data)):
    text = re.sub("[-()\"#/@;:“”‘’<>{}'`+=~|.!?,]", " ", data['text'][i])
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('(https?\://)\S+', '',text) # Remove links
    text = re.sub('(\\n)|\n|\r|\t', '',text) # Remove CR, tab, and LR
    text = re.sub('@([A-Za-z0-9_]+)', '',text) # Remove usernames
    text = re.sub('[0-9]', '',text) # Remove numbers
    text = ' '.join(text for text in text.split() if len(text)>2)
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stop_words]
    text = ' '.join(text)
    corpus.append(text)

In [ ]:
corpus[0]

'washington reuter head conserv republican faction congress vote month huge expans nation debt pay tax cut call fiscal conserv sunday urg budget restraint keep sharp pivot way among republican repres mark meadow speak cb face nation drew hard line feder spend lawmak brace battl januari return holiday wednesday lawmak begin tri pass feder budget fight like link issu immigr polici even novemb congression elect campaign approach republican seek keep control congress presid donald trump republican want big budget increas militari spend democrat also want proport increas non defens discretionari spend program support educ scientif research infrastructur public health environment protect trump administr alreadi will say go increas non defens discretionari spend percent meadow chairman small influenti hous freedom caucu said program democrat say enough need give govern pay rais percent fiscal conserv see rational eventu run peopl money said meadow among republican vote late decemb parti debt 

In [ ]:
#define vocabulary size
voc_size = 10000

#apply one hot encoder
onehot = [one_hot(words,voc_size)for words in corpus] 

In [ ]:
#Padding embedded documents to make the all inputs having the same length
#vectoriaztion using Pre-Sequence Padding
embedded_docs=pad_sequences(onehot, padding='pre', maxlen=5000)
print(embedded_docs)

[[   0    0    0 ... 1499 5307 1562]
 [   0    0    0 ... 7164 4024 9389]
 [   0    0    0 ...  418 9309 4790]
 ...
 [   0    0    0 ... 7909  855 7325]
 [   0    0    0 ... 7909 9070 5370]
 [   0    0    0 ... 7909 7428 7325]]


In [ ]:
#Building LSTM model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=5000))
model.add(Dropout(0.3)) #30% dropout layer to prevent overfitting
model.add(LSTM(100)) #Adding 100 lstm neurons in the layer
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

#Compiling
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5000, 40)          400000    
_________________________________________________________________
dropout (Dropout)            (None, 5000, 40)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),data['target'].shape

(44898, (44898,))

In [ ]:
#Fitting the Mmdel
# Converting the X and y as array
X = np.array(embedded_docs)
y = np.array(data['target'])

#Check shape of X and y final
X.shape,y.shape

((44898, 5000), (44898,))

In [ ]:
# Splitting dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fitting with 10 epochs and 64 batch size
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
562/562 [==============================] - 3125s 6s/step - loss: 0.1748 - accuracy: 0.9342 - val_loss: 0.0793 - val_accuracy: 0.9758
Epoch 2/10
562/562 [==============================] - 3246s 6s/step - loss: 0.0490 - accuracy: 0.9841 - val_loss: 0.0567 - val_accuracy: 0.9842
Epoch 3/10
562/562 [==============================] - 3385s 6s/step - loss: 0.0492 - accuracy: 0.9852 - val_loss: 0.0199 - val_accuracy: 0.9959
Epoch 4/10
562/562 [==============================] - 3386s 6s/step - loss: 0.0128 - accuracy: 0.9965 - val_loss: 0.0150 - val_accuracy: 0.9970
Epoch 5/10
562/562 [==============================] - 3376s 6s/step - loss: 0.0277 - accuracy: 0.9922 - val_loss: 0.0165 - val_accuracy: 0.9955
Epoch 6/10
562/562 [==============================] - 3376s 6s/step - loss: 0.0090 - accuracy: 0.9976 - val_loss: 0.0281 - val_accuracy: 0.9932
Epoch 7/10
562/562 [==============================] - 3376s 6s/step - loss: 0.0283 - accuracy: 0.9903 - val_loss: 0.0276 - val_accuracy:

In [ ]:
#calculate confusion_matrix and classification report
from sklearn import metrics
from sklearn.metrics import classification_report
y_pred=model.predict_classes(X_test)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print("-----------------------------------------------------------------")
print(classification_report(y_test,y_pred))

[[4308   22]
 [  47 4603]]
-----------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4330
           1       1.00      0.99      0.99      4650

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
#calculate the accuracy of LSTM model
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9923162583518931

In [ ]:
#Done By: Mohammed A. M. Salem
#Supervisor: Dr. Motaz Saad